In [28]:
import numpy as np
import matplotlib.pyplot as plt
# import astropy.constants as con
# import astropy.units as u
import random

In [ ]:
VARIABLES

In [29]:
e = 1.6021766e-19
eps0 = 8.854187817e-12
#eps0 = con.eps0
# m_alpha = 6.644657e-27 *u.kg
# c_alpha = 2*e

In [ ]:
CLASS DEFINITIONS

In [30]:
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def locate(self):
      return np.array([self.x,self.y])

    def length(self):
        return np.sqrt((self.x **2) + (self.y **2))
    
    def dot(self, other):
        return (self.x*other.x) + (self.y*other.y)
    
    def angle_between(self, other):
        return np.arccos(self.dot(other) / (self.length()*other.length()) )
                        
    
    def dist_to(self, other):
        return np.sqrt(((other.y - self.y)**2) + ((other.x - self.x)**2))
    def sub(self,other):
      return Vector(self.x-other.x,self.y-other.y)

In [31]:
# Our Test Particle object
class Particle:
  '''
  ~ This Object is the particle that we can place into the system with initial positions and velocities
  pos [Vector object!] - Represents the particle's position in space
  vel [Vector object!] - Represents the particle's velocity 
  acc [Vector object!] - Reprecents the particle's acceleration
  charge [Scalar]- Represents particle charge
  mass [Scalar] - Represents particle mass
  '''
  def __init__(self, pos, vel, acc, charge, mass):
    self.pos = pos
    self.vel = vel
    self.acc = acc
    self.charge = charge
    self.mass=mass

  def move(self, other, dt):
    ''' dt is the time step of the simulation, it dictates the precision of the simulation.    '''

    # x movement
    self.pos.x = self.pos.x + self.vel.x * dt #+ 0.5*self.acc.x*dt
    self.vel.x = self.vel.x + self.acc.x * dt
    
    # y movement
    self.pos.y = self.pos.y + self.vel.y * dt #+ 0.5*self.acc.y*dt
    self.vel.y = self.vel.y + self.acc.y * dt

    # Changing Acceleration
    self.acc.x = ( 1/(4*np.pi* eps0) * (self.charge * other.charge / (self.mass*((self.pos.dist_to(other.pos))**2))) ) * ((self.pos.x-other.pos.x)/(self.pos.sub(other.pos).length())) 
    self.acc.y = ( 1/(4*np.pi* eps0) * (self.charge * other.charge / (self.mass*((self.pos.dist_to(other.pos))**2))) ) * ((self.pos.y-other.pos.y)/(self.pos.sub(other.pos).length()))

In [ ]:
TESTING

In [32]:
electron1 = Particle(Vector(-1,0.2), Vector(50,0), Vector(0,0), e,9.11e-31)
electron2 = Particle(Vector(2,0), Vector(0,0), Vector(0,0), e,9.11e-31)
posx=[]
posy=[]

print([electron1.acc.x, electron1.acc.y])

plt.figure(figsize = (6,3))
plt.xlim(-1,4)
plt.ylim(-1,4)

plt.scatter(electron1.pos.x, electron1.pos.y)
plt.scatter(electron2.pos.x, electron2.pos.y)

for t in range (150000):
  electron1.move(electron2, 1e-6)
  posx.append(electron1.pos.x)
  posy.append(electron1.pos.y)

plt.plot(posx,posy)


# for t in range (1000000):
#   electron2.move(electron1, 0.0000001)
#   posx2.append(electron2.pos.x)
#   posy2.append(electron2.pos.y)
# plt.plot(posx2,posy2)

[0, 0]


In [39]:
import pandas as pd
X=[]
Y=[]
for i in ([0.01,0.03,0.05,0.1,0.2,0.3,0.5]):
  electron1 = Particle(Vector(-1,i), Vector(50,0), Vector(0,0), e,9.11e-31)
  electron2 = Particle(Vector(2,0), Vector(0,0), Vector(0,0), e,9.11e-31)
  posx=[]
  posy=[]

  plt.figure(figsize = (6,3))
  plt.xlim(-1,4)
  plt.ylim(-1,4)

  plt.scatter(electron1.pos.x, electron1.pos.y)
  plt.scatter(electron2.pos.x, electron2.pos.y)

  for t in range (15000):
    electron1.move(electron2, 1e-5)
    if t%100==0:
      posx.append(electron1.pos.x)
      posy.append(electron1.pos.y)
  X.append(posx)
  Y.append(posy)
  plt.plot(posx,posy)
df = pd.DataFrame({'X_0.01': X[0],'Y_0.01': Y[0],'X_0.03': X[1],'Y_0.03': Y[1],
                   'X_0.05': X[2],'Y_0.05': Y[2],'X_0.1': X[3],'Y_0.1': Y[3],
                   'X_0.2': X[4],'Y_0.2': Y[4],'X_0.3': X[5],'Y_0.3': Y[5],
                   'X_0.5': X[6],'Y_0.5': Y[6],})
df.to_csv(index=False)
compression_opts = dict(method='zip',archive_name='testing.csv')  
df.to_csv('testing.zip', index=False,compression=compression_opts)

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import scipy
import scipy.integrate
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FFMpegWriter 
from matplotlib import animation
%config InlineBackend.figure_format='retina' # UNCOMMENT IF USING MAC makes display better %matplotlib osx
%matplotlib osx

In [41]:
file=pd.read_csv("testing.csv")
file
metadata = dict(title='Rutherford Scattering', artist='Matplotlib') 
writer = FFMpegWriter(fps=50, metadata=metadata, bitrate=200000) 
fig = plt.figure(dpi=200)
fig, ax = plt.subplots(figsize=(5,5))


with writer.saving(fig, "Animation.mp4", dpi=200): 
    for i in range(len(file["X_0.01"])):
        ax.clear()
        plt.scatter(file["X_0.01"][i],file["Y_0.01"][i],color="blue")
        ax.plot(file["X_0.01"][:i],file["Y_0.01"][:i],color="blue", alpha=0.5)
        plt.scatter(file["X_0.03"][i],file["Y_0.03"][i],color="red")
        ax.plot(file["X_0.03"][:i],file["Y_0.03"][:i],alpha=0.5,color="red")
        plt.scatter(file["X_0.05"][i],file["Y_0.05"][i],color="orange")
        ax.plot(file["X_0.05"][:i],file["Y_0.05"][:i],alpha=0.5,color="orange")
        plt.scatter(file["X_0.1"][i],file["Y_0.1"][i],color="green")
        ax.plot(file["X_0.1"][:i],file["Y_0.1"][:i],alpha=0.5,color="green")
        plt.scatter(file["X_0.2"][i],file["Y_0.2"][i],color="yellow")
        ax.plot(file["X_0.2"][:i],file["Y_0.2"][:i],alpha=0.5,color="yellow")
        plt.scatter(file["X_0.3"][i],file["Y_0.3"][i],color="gray")
        ax.plot(file["X_0.3"][:i],file["Y_0.3"][:i],alpha=0.5,color="gray")
        plt.scatter(file["X_0.5"][i],file["Y_0.5"][i],color="black")
        ax.plot(file["X_0.5"][:i],file["Y_0.5"][:i],alpha=0.5,color="black")
        plt.scatter(2,0)
        plt.xlim(-1,4)
        plt.ylim(-1,4)
        plt.xlabel("X")
        plt.ylabel("Y")
        plt.title("Rutherford Scattering of Multiple Electrons")
        plt.draw()
        plt.pause(0.00001)
        writer.grab_frame()